In [18]:
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import getpass
from functions import *
from random import shuffle
from random import seed
from graphviz import Digraph
import pydot
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')

### Loading the data

In [19]:
df = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_actions+sophistication_WITHPAUSE_anonymized_DONT_TOUCH_ME.txt')
df_scores = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_data_anonymized.txt')
df["student"] = df["student"].astype('category')
df_scores["student"] = df_scores["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()

def remodel_families(row):
    if row["Family"] == "Build" or row["Family"] == "Revise":  # or row["Family"] == "Extra":
        if row["Outcome"] != "None":
            return "ConstructWithFeedback"
        else:
            return "Construct"
    else:
        return row["Family"]
    
df['Family'] = df.apply (lambda row: remodel_families (row),axis=1)

In [20]:
df.head()

,Activity,Time Stamp,Family,Action,Component,Outcome,#circuits,#loops,#components,#battery,#circuitSwitch,#grabBagResistor,#lightBulb,#resistor,#seriesAmmeter,student,Anon Student Id
0,a1,1363978501425,Construct,add,lightBulb,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10009106,986526943
1,a1,1363978503940,Organize,organizeWorkspace,junction,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10009106,986526943
2,a1,1363978506034,Organize,organizeWorkspace,junction,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10009106,986526943
3,a1,1363978508034,Organize,organizeWorkspace,junction,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10009106,986526943
4,a1,1363978509800,Organize,organizeWorkspace,lightBulb,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10009106,986526943


In [21]:
df_scores.head()

,Wave,scaffolding,how many PhETs in the past?,how comfortable with phets?,p100,used this circuit sim before?,COMPLETEDNESS,PRIORKNOWLEDGE,COMPLEXITY,DIAGRAMS,...,z post t2,T0_PoCC,T1_PoCC,T2_PoCC,Avg_Mast_Or,Perceived Value,Clustergroups,fourgroups,student,Anon Student Id
0,March,scaff,2,2,0,0,1,1,1,0,...,0.33,2.67,2.78,2.67,4.0,3.50,attitude,scaffolded_attitude,1075123,1920143974
1,March,scaff,2,3,0,0,1,0,1,1,...,-0.12,2.44,3.11,3.33,4.5,3.25,attitude,scaffolded_attitude,3591123,-714775387
2,March,not,2,3,0,0,1,1,1,2,...,-0.72,2.22,2.78,2.44,3.5,3.00,attitude,notscaffolded_attitude,3932129,1194576952
3,March,scaff,2,3,1,1,1,1,1,2,...,-1.18,3.00,3.22,3.11,4.5,3.00,attitude,scaffolded_attitude,4762120,-450686147
4,July,not,1,2,1,0,1,0,1,0,...,-0.18,2.67,3.22,2.78,5.0,3.00,knowledge,notscaffolded_knowledge,4936123,-901793583


In [22]:
get_sequence(df,[94792123])

{94792123: ['Start',
  'Construct',
  'Organize',
  'Pause',
  'Construct',
  'Construct',
  'Organize',
  'Construct',
  'Organize',
  'Construct',
  'Construct',
  'Construct',
  'Organize',
  'ConstructWithFeedback',
  'Organize',
  'Organize',
  'Organize',
  'Organize',
  'Organize',
  'ConstructWithFeedback',
  'Construct',
  'Construct',
  'Organize',
  'Construct',
  'Construct',
  'Organize',
  'Construct',
  'Construct',
  'ConstructWithFeedback',
  'ConstructWithFeedback',
  'Construct',
  'Construct',
  'Construct',
  'Construct',
  'Organize',
  'Construct',
  'Construct',
  'ConstructWithFeedback',
  'Interface',
  'Test',
  'Interface',
  'Test',
  'Pause',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Test',
  'Pause'